In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
from backend import crud
from backend import auth
from backend import models
from backend import schemas

from backend.config import settings
from backend.database import database
from backend.database import SessionLocal

<IPython.core.display.Javascript object>

In [3]:
models.User

backend.models.User

<IPython.core.display.Javascript object>

# Update Attribute

In [4]:
db = SessionLocal()

<IPython.core.display.Javascript object>

In [5]:
user = db.query(models.User).filter(models.User.id == 1).first()

<IPython.core.display.Javascript object>

In [7]:
user.username = "jochen"

<IPython.core.display.Javascript object>

In [8]:
user.hashed_password = auth.get_password_hash("foobar")

<IPython.core.display.Javascript object>

In [9]:
db.dirty

IdentitySet([<backend.models.User object at 0x113224250>])

<IPython.core.display.Javascript object>

In [10]:
db.commit()

<IPython.core.display.Javascript object>

In [7]:
blub = models.User(username="blub")

<IPython.core.display.Javascript object>

In [8]:
db.add(blub)

<IPython.core.display.Javascript object>

In [9]:
db.commit()

<IPython.core.display.Javascript object>

# Async DB Access

In [11]:
import databases

database = databases.Database(settings.database_url)

<IPython.core.display.Javascript object>

In [5]:
database

<IPython.core.display.Javascript object>

In [6]:
await database.connect()

<IPython.core.display.Javascript object>

In [7]:
users = await crud.get_async_users(database)

<IPython.core.display.Javascript object>

In [8]:
users

[User(username='jochen', id=1, is_active=True, deployments=[]),
 User(username='blub', id=2, is_active=True, deployments=[]),
 User(username='user1', id=3, is_active=True, deployments=[])]

<IPython.core.display.Javascript object>

In [30]:
help(models.User.__table__.select)

Help on method select in module sqlalchemy.sql.selectable:

select(whereclause=None, **params) method of sqlalchemy.sql.schema.Table instance
    Return a SELECT of this :class:`_expression.FromClause`.
    
    .. seealso::
    
        :func:`_expression.select` - general purpose
        method which allows for arbitrary column lists.



<IPython.core.display.Javascript object>

In [29]:
models.User.metadata

MetaData(bind=None)

<IPython.core.display.Javascript object>

In [40]:
query = models.User.__table__.select().where(models.User.__table__.username == "jochen")

AttributeError: 'Table' object has no attribute 'username'

<IPython.core.display.Javascript object>

In [9]:
await database.query(models.User).filter_by(username="jochen").first()

AttributeError: 'Database' object has no attribute 'query'

<IPython.core.display.Javascript object>

In [39]:
str(query.compile())

'SELECT users.id, users.username, users.hashed_password, users.is_active \nFROM users'

<IPython.core.display.Javascript object>

In [56]:
username = "jochen"
query = f"select * from users where username = :username"
print(query)

select * from users where username = :username


<IPython.core.display.Javascript object>

In [57]:
user = await database.fetch_one(query, values={"username": username})

<IPython.core.display.Javascript object>

In [59]:
dict(user)

{'id': 1,
 'hashed_password': 'foobarnotreallyhashed',
 'is_active': True,
 'username': 'jochen'}

<IPython.core.display.Javascript object>

In [60]:
# db.execute(query, {"username": username})

<IPython.core.display.Javascript object>

In [64]:
schemas.User(**dict(user))

User(username='jochen', id=1, is_active=True, deployments=[])

<IPython.core.display.Javascript object>

In [62]:
user

<IPython.core.display.Javascript object>

# asyncpg

In [8]:
import asyncpg

<IPython.core.display.Javascript object>

In [9]:
conn = await asyncpg.connect(settings.database_url)

<IPython.core.display.Javascript object>

In [10]:
values = await conn.fetch("SELECT * FROM users")

<IPython.core.display.Javascript object>

In [11]:
for value in values:
    print(value)

<Record id=1 hashed_password='foobarnotreallyhashed' is_active=True username='jochen'>


<IPython.core.display.Javascript object>

In [71]:
await conn.close()

<IPython.core.display.Javascript object>

In [73]:
async with await asyncpg.connect(settings.database_url) as conn:
    stmt = await conn.prepare("select * from users where username = $1")
    print(await stmt.fetchval("jochen"))

AttributeError: __aenter__

<IPython.core.display.Javascript object>

In [12]:
conn = await asyncpg.connect(settings.database_url)
stmt = await conn.prepare("select * from users where username = $1")
print(await stmt.fetch("jochen"))
await conn.close()

[<Record id=1 hashed_password='foobarnotreallyhashed' is_active=True username='jochen'>]


<IPython.core.display.Javascript object>

In [20]:
conn = await asyncpg.connect(settings.database_url)
stmt = await conn.prepare("select * from users where username = $1")
row = await stmt.fetchrow("jochen2")
print(row)
await conn.close()

None


<IPython.core.display.Javascript object>

In [16]:
dict(row)

{'id': 1,
 'hashed_password': 'foobarnotreallyhashed',
 'is_active': True,
 'username': 'jochen'}

<IPython.core.display.Javascript object>

In [17]:
row.username

AttributeError: 'asyncpg.Record' object has no attribute 'username'

<IPython.core.display.Javascript object>

In [19]:
row["username"]

'jochen'

<IPython.core.display.Javascript object>

# passlib

In [11]:
from passlib.context import CryptContext

<IPython.core.display.Javascript object>

In [12]:
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

<IPython.core.display.Javascript object>

In [13]:
pwd_context.hash("foobar")

'$2b$12$gdoWjGXy041P5JjBqVtoL.abwvQu9JnE8Fwqd0pmUlOw1E/nMaSUO'

<IPython.core.display.Javascript object>